# import important library.


In [1]:
# pip install -U setuptools pip wheel
# pip install -U chatterbot==1.0.4
# import important modeul 
# sudo pip install numpy python-dateutil pytz pyparsing six --force-reinstall --upgrade

import numpy as np
import pandas as pd

import os
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder as LE
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
import datetime
import joblib

In [2]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


# Read data


In [3]:
data = pd.read_csv('BankFAQs.csv')
data

,Question,Answer,Class
0,What are the documents required for opening a ...,Following documents are required to open a Cur...,accounts
1,Can I transfer my Current Account from one bra...,"Yes, Current Accounts can be transferred from ...",accounts
2,My present status is NRI. What extra documents...,NRI/PIO can open the proprietorship/partnershi...,accounts
3,What are the documents required for opening a ...,Following documents are required for opening a...,accounts
4,What documents are required to change the addr...,Following documents are required to change the...,accounts
...,...,...,...
1768,Which languages do you speak,just English. But I'm trying to learn new lang...,accounts
1769,how much is interest for FD,Seven Percentage,accounts
1770,how much is interest for Fixed Deposit,Seven Percentage,accounts
1771,Fixed Deposit,Seven Percentage,accounts


In [4]:
joblib.dump(data,'/content/drive/MyDrive/chat_bot_deploay/Bank_data')

['/content/drive/MyDrive/chat_bot_deploay/Bank_data']

In [5]:
le = LE()

In [6]:
questions = data['Question'].values

In [7]:
data.Class.unique()

array(['accounts', 'cards', 'fundstransfer', 'insurance', 'investments',
       'loans', 'security'], dtype=object)

# Data cleaning 

In [8]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
  
stop_words = set(stopwords.words('english'))
print(stop_words)

stemmer = LancasterStemmer()

def cleanup(sentence):
  word_tok = nltk.word_tokenize(sentence)
  stemmed_words = [w for w in word_tok if not w in stop_words]
  print(stemmed_words)
  return '  '.join(stemmed_words)

x = []
for que in questions:
  x.append(cleanup(que))


{'having', 'there', 'your', 'she', 'own', 's', 'until', 'into', 'aren', 'mustn', 'its', 'hasn', 'him', 'yourself', 'about', "weren't", 'other', 'hadn', "mightn't", "wouldn't", 'a', 'ma', 'that', 'we', 'he', 'ourselves', 'have', 'should', 'our', 'mightn', "shan't", 'as', 'no', 'my', 'was', 'up', 'from', 'don', 'it', 'than', "doesn't", 'before', 'theirs', 'whom', "hasn't", 'me', 'so', 'then', "needn't", "should've", 'this', 'between', "that'll", 'some', 'didn', "didn't", 'her', 'nor', 'below', 'i', 'more', 'or', 'does', "couldn't", 'herself', 'them', 'haven', "you'd", 'won', "wasn't", 'further', 'an', 'couldn', 'what', 'y', 'doesn', 'for', 'off', 'through', 'be', 'll', 'itself', 'all', 'any', 'weren', "hadn't", 'his', 'both', 'at', 'their', "it's", "you've", 'such', 'here', 'has', 'o', 'myself', 'too', "isn't", 'just', 'm', 'can', 'ours', 'those', 'while', 'the', 'under', 'd', 'down', "don't", 'shouldn', 'once', 'being', 'out', 'again', 'needn', 'did', "you're", 'when', 'where', 'of', 'i

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['What', 'transactions', 'included', 'part', 'Free', 'Anywhere', 'Transactions', 'limit', 'month']
['Can', 'I', 'make', 'payments', 'suppliers', 'M', '&', 'N', 'Bank', 'branches', 'single', 'cheque', 'book']
['What', 'permissible', 'Debits', 'RFC', 'Domestic', 'Account']
['Can', 'I', 'access', 'account', 'telephone', 'conduct', 'routine', 'daily', 'transactions']
['Who', 'open', 'RFC', 'Domestic', 'Account']
['What', 'documents', 'required', 'opening', 'Current', 'Account', 'partnership', 'firm']
['What', 'permissible', 'credits', 'RFC', 'Domestic', 'Account']
['In', 'currencies', 'I', 'open', 'RFC', 'Domestic', 'account']
['Whether', 'EEFC', 'balances', 'covered', 'exchange', 'risk']
['Is', 'restriction', 'withdrawal', 'rupees', 'funds', 'held', 'EEFC', 'account']
['How', 'much', 'foreign', 'exchange', 'earnings', 'credited', 'EEFC', 'account']
['What', 'permissible', 'credits', 'EEFC', 'account']
['What', 'permissible', 'debits', 'account']
['What', 'benefits', 'Smartup', 'Solutions'

In [9]:
tfv = TfidfVectorizer(min_df=1,stop_words='english')

In [10]:
tfv.fit(x)

TfidfVectorizer(stop_words='english')

In [11]:
x = tfv.transform(x)

In [12]:
le.fit(data['Class'])


LabelEncoder()

In [13]:
y = le.transform(data['Class'])
y

array([0, 0, 0, ..., 0, 0, 0])

# using smote oversampling 


In [14]:
# from imblearn.over_sampling import SMOTE
# import numpy as np
# smote = SMOTE()
# x_res,y_res = smote.fit_resample(x,y)
# print(np.count_nonzero(y_res == 0))
# print(np.count_nonzero(y_res == 1))
# print(np.count_nonzero(y_res == 2))
# print(np.count_nonzero(y_res == 3))
# print(np.count_nonzero(y_res == 4))
# print(np.count_nonzero(y_res == 5))
# print(np.count_nonzero(y_res == 6))

In [15]:
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=58,test_size=0.2,stratify=y)

In [16]:
from sklearn.metrics import r2_score
model = SVC(kernel = 'linear')
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
accuracy = r2_score(y_test,y_pred)*100
print(f"Accuracy is = {accuracy}")

Accuracy is = 88.20640080623224


# Model dump using joblib


In [17]:
joblib.dump(tfv,'/content/drive/MyDrive/chat_bot_deploay/tfidf_')

['/content/drive/MyDrive/chat_bot_deploay/tfidf_']

In [18]:
joblib.dump(x,'/content/drive/MyDrive/chat_bot_deploay/question_vectorizer')

['/content/drive/MyDrive/chat_bot_deploay/question_vectorizer']